In [1]:
import os
import datetime
import helper as hp
from helper import Box, GFX, Font, Pin, Property, PinName, mils_to_units

In [2]:
out_dir = 'Created'
symbol_name = 'test_symbol'

# Defaults
symbol_type = 'Module'

In [3]:
f = open(os.path.join(out_dir, symbol_name) + '.1', 'w')

In [4]:
def create_main_hdr(f):
    hdr = ['V 54']
    hdr += ['K 33671749690 new_symbol']
    hdr += ['F Case']
    hdr += ['|R ' + datetime.datetime.now().strftime('%H:%M:%S_%m-%d-%y')]
    hdr += ['|BORDERTYPESUPPORT']
    hdr += ['Y ' + str(hp.symtype_to_idx[symbol_type])]
    hdr += ['D 0 0 2540000 2540000', 'Z 10', 'i 3', '|I 6']

    write_str_list(f, hdr)

def write_str_list(f, str_list):
    for s in str_list:
        f.write(s)
        f.write('\n')
        
def add_pin(f, pid, x, y, side, inv, num, label, pin_type):
    str_list = []
    
    pin = Pin()
    pin.simple_pin(pid, x, y, side, inv, pin_type, label, num)
    str_list += [pin.get_str()]
        
    str_list += [pin.Name.get_str()]
    str_list += [Font().create_str('Sans Serif', 'Dark Blue')]
    
    str_list += [pin.get_pintype_str()]
    
    str_list += [pin.Number.get_str()]
    str_list += [Font().create_str('Sans Serif', 'Automatic')]
    
    write_str_list(f, str_list)    

def add_font(f, font, color):
    vals = [Font().create_str(font, color)]
    write_str_list(f, vals)

def add_graphics(hdr, color, fill_style, line_style, w):
    color = hp.color_to_int(color)
    vals = [GFX().create_str(hdr, color, color, fill_style, line_style, 1)]
    write_str_list(f, vals)

def add_box(f, x, y, w, h):
    x2 = x + w
    y2 = y + h
    vals = [Box().create_str(mils_to_units(x), mils_to_units(y), mils_to_units(x2), mils_to_units(y2))]
    write_str_list(f, vals)
    
    add_graphics('|GRPHSTL_EXT01', 'Blue', 'Hollow', 'Solid', 1)

In [5]:
create_main_hdr(f)
add_pin(f, 1, 0, 0, 'Left', False, 48, 'RESET', 'OUT')
add_box(f, 300, -100, 1200, 2500)
f.write('E')
f.close()